## Solving Sequence Problems with LSTM in Keras: Part 2
<a href="https://stackabuse.com/solving-sequence-problems-with-lstm-in-keras-part-2/">Tutorial Link</a>

This is the second and final part of the two-part series of articles on solving sequence problems with LSTMs. In the part 1 of the series, I explained how to solve one-to-one and many-to-one sequence problems using LSTM. In this part, you will see how to solve one-to-many and many-to-many sequence problems via LSTM in Keras.

Image captioning is a classic example of one-to-many sequence problems where you have a single image as input and you have to predict the image description in the form of a word sequence. Similarly, stock market prediction for the next X days, where input is the stock price of the previous Y days, is a classic example of many-to-many sequence problems.

In this article you will see very basic examples of one-to-many and many-to-many problems. However, the concepts learned in this article will lay the foundation for solving advanced sequence problems, such as stock price prediction and automated image captioning that we will see in the upcoming articles.
    
## One-to-Many Sequence Problems
    
One-to-many sequence problems are the type of sequence problems where input data has one time-step and the output contains a vector of multiple values or multiple time-steps. In this section, we will see how to solve one-to-many sequence problems where the input has a single feature. We will then move on to see how to work with multiple features input to solve one-to-many sequence problems.
    
## One-to-Many Sequence Problems with a Single Feature
Let's first create a dataset and understand the problem that we are going to solve in this section.

Creating the Dataset
The following script imports the required libraries:

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from tensorflow.keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
# GPU Temp Settings
tf.disable_v2_behavior()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)
config = tf.ConfigProto(gpu_options=gpu_options)
config.gpu_options.allow_growth=True

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
X = list()
Y = list()
X = [x+3 for x in range(-2, 43, 3)]

for i in X:
    output_vector = list()
    output_vector.append(i+1)
    output_vector.append(i+2)
    Y.append(output_vector)

print(X)
print(Y)

[1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43]
[[2, 3], [5, 6], [8, 9], [11, 12], [14, 15], [17, 18], [20, 21], [23, 24], [26, 27], [29, 30], [32, 33], [35, 36], [38, 39], [41, 42], [44, 45]]


Our input contains 15 samples with one time-step and one feature value. For each value in the input sample, the corresponding output vector contains the next two integers. For instance, if the input is 4, the output vector will contain values 5 and 6. Hence, the problem is a simple one-to-many sequence problem.

The following script reshapes our data as required by the LSTM:

In [3]:
X = np.array(X).reshape(len(X), 1, 1)
Y = np.array(Y)

## Solution via Simple LSTM

In [6]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3)

Train on 12 samples, validate on 3 samples
Epoch 1/1000
12/12 [==============================] - 0s 4ms/sample - loss: 466.8390 - val_loss: 1696.0038
Epoch 2/1000
12/12 [==============================] - 0s 605us/sample - loss: 458.1834 - val_loss: 1671.2789
Epoch 3/1000
12/12 [==============================] - 0s 648us/sample - loss: 451.8585 - val_loss: 1646.0684
Epoch 4/1000
12/12 [==============================] - 0s 520us/sample - loss: 444.9642 - val_loss: 1621.2593
Epoch 5/1000
12/12 [==============================] - 0s 644us/sample - loss: 436.5850 - val_loss: 1598.0878
Epoch 6/1000
12/12 [==============================] - 0s 518us/sample - loss: 429.4555 - val_loss: 1575.5468
Epoch 7/1000
12/12 [==============================] - 0s 691us/sample - loss: 422.8487 - val_loss: 1553.1685
Epoch 8/1000
12/12 [==============================] - 0s 557us/sample - loss: 416.5646 - val_loss: 1529.2008
Epoch 9/1000
12/12 [==============================] - 0s 461us/sample - loss: 407.8102 

Once the model is trained we can make predictions on the test data:

In [7]:
test_input = array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input, verbose=0)
print(f"Actual: (11, 12) || Prediction: {test_output}")

Actual: (11, 12) || Prediction: [[10.946018 12.057393]]


The test data contains a value 10. In the output, we should get a vector containing 11 and 12. The output I received is [10.946018 12.057393] which is actually very close to the expected output.

## Solution via Stacked LSTM
The following script trains stacked LSTMs on our data and makes prediction on the test points:

In [10]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Train on 12 samples, validate on 3 samples
Epoch 1/1000
12/12 [==============================] - 0s 10ms/sample - loss: 470.7038 - val_loss: 1734.5045
Epoch 2/1000
12/12 [==============================] - 0s 832us/sample - loss: 468.8572 - val_loss: 1728.1691
Epoch 3/1000
12/12 [==============================] - 0s 763us/sample - loss: 467.3559 - val_loss: 1722.0172
Epoch 4/1000
12/12 [==============================] - 0s 679us/sample - loss: 465.3518 - val_loss: 1716.3502
Epoch 5/1000
12/12 [==============================] - 0s 883us/sample - loss: 463.8554 - val_loss: 1710.0258
Epoch 6/1000
12/12 [==============================] - 0s 743us/sample - loss: 461.8508 - val_loss: 1702.1002
Epoch 7/1000
12/12 [==============================] - 0s 622us/sample - loss: 459.5604 - val_loss: 1691.6826
Epoch 8/1000
12/12 [==============================] - 0s 747us/sample - loss: 456.5469 - val_loss: 1676.1879
Epoch 9/1000
12/12 [==============================] - 0s 709us/sample - loss: 451.6586

In [11]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: (11, 12) || Prediction: {test_output}")

Actual: (11, 12) || Prediction: [[11.028526 12.002527]]


The answer is [11.028526 12.002527] which is very close to the actual output.

## Solution via Bidirectional LSTM
The following script trains a bidirectional LSTM on our data and then makes a prediction on the test set.

In [14]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(1, 1)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Train on 12 samples, validate on 3 samples
Epoch 1/1000
12/12 [==============================] - 0s 14ms/sample - loss: 469.4308 - val_loss: 1707.2111
Epoch 2/1000
12/12 [==============================] - 0s 650us/sample - loss: 457.4735 - val_loss: 1665.0385
Epoch 3/1000
12/12 [==============================] - 0s 602us/sample - loss: 445.0269 - val_loss: 1625.3473
Epoch 4/1000
12/12 [==============================] - 0s 559us/sample - loss: 436.1325 - val_loss: 1583.2700
Epoch 5/1000
12/12 [==============================] - 0s 560us/sample - loss: 425.1626 - val_loss: 1542.1627
Epoch 6/1000
12/12 [==============================] - 0s 615us/sample - loss: 413.8366 - val_loss: 1500.8539
Epoch 7/1000
12/12 [==============================] - 0s 771us/sample - loss: 401.7686 - val_loss: 1459.5599
Epoch 8/1000
12/12 [==============================] - 0s 899us/sample - loss: 390.2377 - val_loss: 1415.9603
Epoch 9/1000
12/12 [==============================] - 0s 944us/sample - loss: 378.3119

In [15]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: (11, 12) || Prediction: {test_output}")

Actual: (11, 12) || Prediction: [[10.9696045 12.050887 ]]


## One-to-Many Sequence Problems with Multiple Features
In this section we will see one-to-many sequence problems where input samples will have one time-step, but two features. The output will be a vector of two elements.

## Creating the Dataset
As always, the first step is to create the dataset:

In [16]:
nums = 25

X1 = list()
X2 = list()
X = list()
Y = list()

X1 = [(x+1)*2 for x in range(25)]
X2 = [(x+1)*3 for x in range(25)]

for x1, x2 in zip(X1, X2):
    output_vector = list()
    output_vector.append(x1+1)
    output_vector.append(x2+1)
    Y.append(output_vector)

X = np.column_stack((X1, X2))
print(X)

[[ 2  3]
 [ 4  6]
 [ 6  9]
 [ 8 12]
 [10 15]
 [12 18]
 [14 21]
 [16 24]
 [18 27]
 [20 30]
 [22 33]
 [24 36]
 [26 39]
 [28 42]
 [30 45]
 [32 48]
 [34 51]
 [36 54]
 [38 57]
 [40 60]
 [42 63]
 [44 66]
 [46 69]
 [48 72]
 [50 75]]


You can see each input time-step consists of two features. The output will be a vector which contains the next two elements that correspond to the two features in the time-step of the input sample. For instance, for the input sample [2, 3], the output will be [3, 4], and so on.

Let's reshape our data:

In [17]:
X = np.array(X).reshape(25, 1, 2)
Y = np.array(Y)

## Solution via Simple LSTM

In [18]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 2)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
model.fit(X, Y, epochs=1000, validation_split=0.2, batch_size=3)

Train on 20 samples, validate on 5 samples
Epoch 1/1000
20/20 [==============================] - 0s 8ms/sample - loss: 977.8858 - val_loss: 3419.0904
Epoch 2/1000
20/20 [==============================] - 0s 652us/sample - loss: 921.8774 - val_loss: 3198.5357
Epoch 3/1000
20/20 [==============================] - 0s 642us/sample - loss: 865.1943 - val_loss: 2984.7384
Epoch 4/1000
20/20 [==============================] - 0s 693us/sample - loss: 801.6414 - val_loss: 2757.7044
Epoch 5/1000
20/20 [==============================] - 0s 745us/sample - loss: 733.5690 - val_loss: 2476.7200
Epoch 6/1000
20/20 [==============================] - 0s 789us/sample - loss: 654.7643 - val_loss: 2142.9848
Epoch 7/1000
20/20 [==============================] - 0s 663us/sample - loss: 562.7139 - val_loss: 1763.5187
Epoch 8/1000
20/20 [==============================] - 0s 655us/sample - loss: 461.9083 - val_loss: 1363.8251
Epoch 9/1000
20/20 [==============================] - 0s 711us/sample - loss: 353.4449 

Let's now create our test point and see how well our algorithm performs:

In [19]:
test_input = array([40, 60])
test_input = test_input.reshape((1, 1, 2))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[41.00864 61.04534]]


The input is [40, 60], the output should be [41, 61]. The output predicted by our simple LSTM is [41.00864 61.04534] which is very close to the expected output.
## Solution via Stacked LSTM

In [20]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 2)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Train on 20 samples, validate on 5 samples
Epoch 1/1000
20/20 [==============================] - 0s 11ms/sample - loss: 985.9319 - val_loss: 3552.5704
Epoch 2/1000
20/20 [==============================] - 0s 915us/sample - loss: 976.3882 - val_loss: 3514.2993
Epoch 3/1000
20/20 [==============================] - 0s 928us/sample - loss: 965.8249 - val_loss: 3446.5383
Epoch 4/1000
20/20 [==============================] - 0s 886us/sample - loss: 946.8322 - val_loss: 3335.2532
Epoch 5/1000
20/20 [==============================] - 0s 909us/sample - loss: 914.3898 - val_loss: 3170.7410
Epoch 6/1000
20/20 [==============================] - 0s 837us/sample - loss: 852.2999 - val_loss: 2899.4584
Epoch 7/1000
20/20 [==============================] - 0s 871us/sample - loss: 753.6391 - val_loss: 2428.9381
Epoch 8/1000
20/20 [==============================] - 0s 848us/sample - loss: 599.4738 - val_loss: 1644.4887
Epoch 9/1000
20/20 [==============================] - 0s 748us/sample - loss: 365.0485

In [21]:
test_input = array([40, 60])
test_input = test_input.reshape((1, 1, 2))
test_output = model.predict(test_input, verbose=0)
print(f"Actual: [41, 61) || Prediction: {test_output}")

Actual: [41, 61) || Prediction: [[41.11495  61.205273]]


## Solution via Bidirectional LSTM

In [22]:
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(1, 2)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

history = model.fit(X, Y, epochs=1000, validation_split=0.2, verbose=1, batch_size=3)

Train on 20 samples, validate on 5 samples
Epoch 1/1000
20/20 [==============================] - 0s 12ms/sample - loss: 857.2945 - val_loss: 2946.4588
Epoch 2/1000
20/20 [==============================] - 0s 913us/sample - loss: 791.2904 - val_loss: 2712.9641
Epoch 3/1000
20/20 [==============================] - 0s 737us/sample - loss: 731.9177 - val_loss: 2450.6070
Epoch 4/1000
20/20 [==============================] - 0s 817us/sample - loss: 655.2766 - val_loss: 2148.1716
Epoch 5/1000
20/20 [==============================] - 0s 779us/sample - loss: 564.9252 - val_loss: 1809.1647
Epoch 6/1000
20/20 [==============================] - 0s 792us/sample - loss: 465.0822 - val_loss: 1433.7393
Epoch 7/1000
20/20 [==============================] - 0s 820us/sample - loss: 377.9405 - val_loss: 1017.2776
Epoch 8/1000
20/20 [==============================] - 0s 861us/sample - loss: 257.4260 - val_loss: 631.0074
Epoch 9/1000
20/20 [==============================] - 0s 757us/sample - loss: 148.6083 

In [23]:
test_output = model.predict(test_input, verbose=0)
print(f"Actual: [41, 61) || Prediction: {test_output}")

Actual: [41, 61) || Prediction: [[41.01211  61.004776]]
